In [ ]:
pip install pysal

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import libpysal
from esda import Moran
import os

In [13]:
input_directory = r'G:\Official_Vanda_Organizing\Spatial_Morphologic_Data\2_Data_Cleaning\4_Pinnacle_Volumes'
output_directory = r'G:\Official_Vanda_Organizing\Spatial_Morphologic_Data\2_Data_Cleaning\4_Pinnacle_Volumes\Height_Volume_Ratio_MoransI'

# Ensure output directory exists
os.makedirs(output_directory, exist_ok=True)

# Loop over each CSV file in the input directory
for file_name in os.listdir(input_directory):
    if file_name.endswith('.csv'):
        file_path = os.path.join(input_directory, file_name)
        
        # Load the data
        data = pd.read_csv(file_path)

        # Convert height to cm and calculate height/volume ratio
        data['Height_cm'] = data['Height_m'] * 100
        data['HeightVolRatio'] = data['Height_cm'] / data['Volume_cm3']

        # Extract coordinates and the ratio
        x_coords = data['MaxX']
        y_coords = data['MaxY']
        ratios = data['HeightVolRatio']

        coordinates = np.array(list(zip(x_coords, y_coords)))

        try:
            # Create spatial weights matrix (distance-based, threshold in coordinate units)
            w = libpysal.weights.DistanceBand(coordinates, threshold=.05, binary=True, silence_warnings=True)

            # Calculate Moran's I
            moran = Moran(ratios, w)

            # Print results
            print(f"Processing {file_name}:")
            print(f"Moran's I: {moran.I:.4f}, P-value: {moran.p_sim:.4f}")

            # Visualize the results
            plt.figure(figsize=(8, 6))
            sc = plt.scatter(x_coords, y_coords, c=ratios, cmap='plasma', marker='o')
            plt.colorbar(sc, label='Height(cm)/Volume(cm³) Ratio')

            plt.suptitle(file_name, fontsize=16)
            plt.title(f"Moran's I: {moran.I:.4f} (p-value: {moran.p_sim:.4f})", fontsize=12)

            plt.xlabel('MaxX')
            plt.ylabel('MaxY')
            plt.grid(True)

            # Save the figure
            output_file_path = os.path.join(output_directory, f"{os.path.splitext(file_name)[0]}.png")
            plt.savefig(output_file_path, format='png', bbox_inches='tight')
            plt.close()
        
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

print("All figures saved successfully.")

Processing C12_11.10to14.24_scaled.csv:
Moran's I: 0.2183, P-value: 0.0010
Processing C12_8.09to11_scaled.csv:
Moran's I: 0.0497, P-value: 0.0450
Processing C4_15.31to19.37_scaled.csv:
Moran's I: 0.1195, P-value: 0.0150
Processing C7_3.21to5.30_scaled_colors_cleaned.csv:
Moran's I: 0.1250, P-value: 0.0050
Processing C8_5.35to7.46_scaled_colors_scaled.csv:
Moran's I: 0.0281, P-value: 0.0500
Processing C9_3.35to6.18_scaled_colors_cleaned.csv:
Moran's I: 0.1644, P-value: 0.0100
Processing C9_6.34to9.46_scaled.csv:
Moran's I: 0.1593, P-value: 0.0010
All figures saved successfully.
